In [ ]:
import os
import numpy as np
import pandas as pd

# Set-up Functions and Variables

create list of Ashkenazi Samples

In [4]:
reich_samples = set()
with open("../ashkenazi_data/EuropeFullyPublic/original_eigenstrat/vdata.ind") as f:
    for line in f.readlines():
        sample = line.strip().split()[0]
        reich_samples.add(sample)

FileNotFoundError: [Errno 2] No such file or directory: '../ashkenazi_data/EuropeFullyPublic/original_eigenstrat/vdata.ind'

In [ ]:
def get_samples_in_cohort(fam_file):
    samples = dict()
    with open(fam_file) as f:
        for line in f.readlines():
            split_line = line.split()
            fam_id = split_line[0]
            sample = split_line[1]
            samples[sample] = fam_id
    return samples


def write_samples_to_include(fam_file, cohort_dir):
    samples = get_samples_in_cohort(fam_file)
    samples_in_cohort = set(samples.keys())

    samples_to_consider = all_sample_list - kg_and_hgdp_sample_list

    if not os.path.basename(cohort_dir) == "EuropeFullyPublic":
        samples_to_consider -= reich_samples

    samples_to_include = samples_in_cohort.intersection(samples_to_consider)
    samples_to_include -= kg_and_hgdp_sample_list

    samples_to_include_filename = cohort_dir + "/samples_to_include.txt"
    samples_output = [[samples[sample], sample] for sample in samples_to_include]
    np.savetxt(samples_to_include_filename, samples_output, fmt="%s", delimiter="\t")
    return samples_to_include_filename


def liftover_vcf_file(cohort):
    input_vcf = cohort.get_suffix_bfile("filtered") + ".vcf"
    mt = hl.import_vcf(input_vcf)
    mt = mt.annotate_rows(new_locus=hl.liftover(mt.locus, 'GRCh38', include_strand=True), old_locus=mt.locus)

    #save not_lifted vcf info
    not_lifted_mt = mt.filter_rows(~hl.is_defined(mt.new_locus) | mt.new_locus.is_negative_strand)
    not_lifted_vcf = cohort.get_suffix_bfile("not_lifted") + ".vcf"
    hl.export_vcf(not_lifted_mt, not_lifted_vcf)

    lifted_mt = mt.filter_rows(hl.is_defined(mt.new_locus) & ~mt.new_locus.is_negative_strand)
    lifted_mt = lifted_mt.key_rows_by(locus=lifted_mt.new_locus.result, alleles=lifted_mt.alleles)
    lifted_mt_filename = cohort.get_suffix_bfile("lifted_mt") + ".vcf"
    hl.export_vcf(lifted_mt, lifted_mt_filename)


class Cohort:

    def __init__(self, original_bfile, is_bfile=True):

        self.original_bfile = original_bfile
        self.original_bfile_name = os.path.basename(self.original_bfile)

        split_dir = self.original_bfile.split("/")
        self.dir = "/".join(split_dir[0:-2])

        if is_bfile:

            fam_file = original_bfile + ".fam"
            samples_to_include_filename = write_samples_to_include(fam_file, self.dir)
            self.samples_to_include_filename = samples_to_include_filename


    def get_suffix_bfile(self, suffix):
        corrected_dir = self.dir + "/" + suffix

        if not os.path.exists(corrected_dir):
            os.mkdir(corrected_dir)

        corrected_bfile = corrected_dir + "/" + self.original_bfile_name + "_" + suffix
        return corrected_bfile

In [ ]:
original_bfiles = ["../ashkenazi_data/caucasus/original_plink/caucasus_paper_data_dbSNP-b131_pos-b37_1KG_strand",
                   "../ashkenazi_data/EuropeFullyPublic/convertf_plink/vdata"]
cohorts = [Cohort(original_bfile) for original_bfile in original_bfiles]

In [ ]:
hg19_fasta = "../../hg38_reference_data/Homo_sapiens_assembly19.fasta"
for i, cohort in enumerate(cohorts):
    print(i)
    !plink2 --bfile {cohort.original_bfile} --ref-from-fa $hg19_fasta --make-pgen \
    --keep {cohort.dir}/samples_to_include.txt --out {cohort.get_suffix_bfile("fixed_ref")}
    !plink2 --pfile {cohort.get_suffix_bfile("fixed_ref")} --sort-vars --make-pgen --out {cohort.get_suffix_bfile("sorted")}
    !plink2 --pfile {cohort.get_suffix_bfile("sorted")} --mind 0.05 --geno 0.05 --export vcf id-paste=iid \
    --out {cohort.get_suffix_bfile("filtered")}
    print()

# Liftover Bed Files

In [ ]:
import hail as hl

In [ ]:
hl.init()

In [ ]:
rg37 = hl.get_reference('GRCh37')
rg38 = hl.get_reference('GRCh38')
rg37.add_liftover("grch37_to_grch38.over.chain.gz", rg38)

In [ ]:
for cohort in cohorts:
    liftover_vcf_file(cohort)